 # 4.2 Volatility Spillover Measures Using Generalized Variance Decomposition (GVD)



 Diebold and Yilmaz (2009, 2012) propose measuring volatility spillovers using the forecast error variance decomposition from a vector autoregressive (VAR) model.



 The VAR($p$) model is specified as:



 $$

 Y_t = \sum_{i=1}^p \Phi_i Y_{t-i} + \varepsilon_t,

 $$



 where $Y_t$ is an $N \times 1$ vector of realized variances, $\Phi_i$ are coefficient matrices, and $\varepsilon_t$ is a white noise error vector with covariance matrix $\Sigma$.



 The moving average representation is:



 $$

 Y_t = \sum_{i=0}^\infty A_i \varepsilon_{t-i}

 $$



 where matrices $A_i$ satisfy the recursion $A_i = \sum_{j=1}^p \Phi_j A_{i-j}$, with $A_0 = I_N$.



The forecast error variance decomposition quantifies the fraction of the $H$-step-ahead forecast error variance of variable $i$ attributable to shocks in variable $j$. The **Generalized Variance Decomposition (GVD)** method by Koop et al. (1996) and Pesaran and Shin (1998) is used because it is invariant to variable ordering.



 The GVD element is:



 $$

\theta^{g}_{ij}(H) = \frac{\sigma_{jj}^{-1} \sum_{h=0}^{H-1} \left( e_i' A_h \Sigma e_j \right)^2 }{

\sum_{h=0}^{H-1} \left( e_i' A_h \Sigma A_h' e_i \right) }

$$



 where $\sigma_{jj}$ is the variance of the error term for equation $j$, and $e_i$ is a selection vector with 1 in position $i$, zero elsewhere.



 Normalizing rows so their sum equals one gives:



$$

\tilde{\theta}^{g}_{ij}(H) = \frac{\theta^{g}_{ij}(H)}{\sum_{j=1}^N \theta^{g}_{ij}(H)}.

$$



 The **Total Spillover Index (TSI)** measures the contribution of spillovers to total forecast error variance:



$$

S^{g}(H) = \frac{\sum_{i=1}^N \sum_{j=1, j\neq i}^N \tilde{\theta}^{g}_{ij}(H)}{N} \times 100.

$$



Directional spillovers transmitted from market $i$ to others and received by market $i$ from others can also be computed from normalized GVD elements.

 ## Step 1: Load PIT-transformed realized variances and prepare VAR input

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

pit_vars = pd.read_parquet("parquet_files/pit_transformed_variances.parquet")
pit_vars.index = pd.to_datetime(pit_vars.index)

# Inspect the data
pit_vars.head()


Area,BZN|ES,BZN|FR,BZN|PT
2021-05-21,-1.228412,-0.469873,-1.221048
2021-05-22,-1.374370,-0.681604,-1.352340
2021-05-23,-1.322559,-0.425643,-1.289900
2021-05-24,-1.239586,-0.541040,-1.210123
2021-05-25,-1.747794,-1.202919,-1.747794


 ## Step 2: Fit VAR($p$) model on realized variances



 Select lag order $p$ (e.g., 1 to 5) based on criteria or domain knowledge.

 Here we use $p=1$ for simplicity.

In [3]:
p = 1
model = VAR(pit_vars)
results = model.fit(p)

print(results.summary())


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 28, May, 2025
Time:                     18:26:13
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -4.74525
Nobs:                     1440.00    HQIC:                  -4.77278
Log likelihood:          -2669.60    FPE:                 0.00831926
AIC:                     -4.78918    Det(Omega_mle):      0.00825031
--------------------------------------------------------------------
Results for equation BZN|ES
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             0.001765         0.019242            0.092           0.927
L1.BZN|ES         0.223558         0.100056            2.234           0.025
L1.BZN|FR         0.093474         0.022686            4.120           0.00

C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


 ## Step 3: Compute moving average coefficient matrices $A_h$



 Calculate matrices $A_h$ for $h=0,\dots,H-1$, using the recursion:



$$

A_0 = I_N, \quad A_h = \sum_{j=1}^p \Phi_j A_{h-j} \quad \text{for } h>0,

$$



 where $\Phi_j$ are the VAR coefficient matrices.

In [4]:
H = 10  # forecast horizon

N = pit_vars.shape[1]
I = np.eye(N)
A = [I]

Phi = results.coefs  # shape (p, N, N)

for h in range(1, H):
    A_h = np.zeros((N,N))
    for j in range(1, min(h,p)+1):
        A_h += Phi[j-1] @ A[h-j]
    A.append(A_h)


 ## Step 4: Compute Generalized Variance Decomposition matrix $\Theta^g(H)$

In [5]:
Sigma = results.sigma_u  # covariance matrix of residuals
sigma_diag_inv = np.diag(1 / np.diag(Sigma))

theta_g = np.zeros((N,N))

for i in range(N):
    e_i = np.zeros(N)
    e_i[i] = 1
    denom = 0
    for h in range(H):
        Ah = A[h]
        denom += e_i @ Ah @ Sigma @ Ah.T @ e_i
    for j in range(N):
        e_j = np.zeros(N)
        e_j[j] = 1
        numer = 0
        for h in range(H):
            Ah = A[h]
            val = e_i @ Ah @ Sigma @ e_j
            numer += val ** 2
        theta_g[i,j] = sigma_diag_inv[j,j] * numer / denom


 ## Step 5: Normalize the variance decomposition matrix row-wise

In [6]:
theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)


 ## Step 6: Compute Total Spillover Index (TSI)

In [7]:
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / N
print(f"Total Spillover Index (TSI) at horizon {H}: {TSI:.2f}%")


Total Spillover Index (TSI) at horizon 10: 46.29%


 ## Step 7: Compute directional spillovers transmitted and received by each market



 - Spillovers transmitted from market $i$:



$$

S^{g}_{.i}(H) = \frac{1}{N} \sum_{j=1, j \neq i}^N \tilde{\theta}^{g}_{ji}(H) \times 100

$$



- Spillovers received by market $i$:



$$

S^{g}_{i.}(H) = \frac{1}{N} \sum_{j=1, j \neq i}^N \tilde{\theta}^{g}_{ij}(H) \times 100

$$

In [8]:
spillovers_transmitted = 100 / N * (theta_g_normalized.T.sum(axis=0) - np.diag(theta_g_normalized))
spillovers_received = 100 / N * (theta_g_normalized.sum(axis=1) - np.diag(theta_g_normalized))

spillovers_df = pd.DataFrame({
    "Transmitted": spillovers_transmitted,
    "Received": spillovers_received
}, index=pit_vars.columns)

print(spillovers_df)


        Transmitted   Received
Area                          
BZN|ES    18.093171  18.093171
BZN|FR    10.282341  10.282341
BZN|PT    17.918171  17.918171


 ## Save spillover measures

In [9]:
spillovers_df.to_parquet("parquet_files/volatility_spillovers.parquet")
